# Modeling solar panel surfaces for space stations

#### By Barthélemy Paléologue

The goal of this document is to create a model of the solar panel area needed to power a space station. With this model, it will be possible to compute the area needed for Cosmos Journeyer's space stations. Multiples factors will be considered, such as the distance between the station and its main star, the station population, the energy consumption per person, the efficiency of the solar panels, and the energy storage capacity. To check the calculations, I will try to calculate the area needed for the international space station (ISS) and compare it with the real data.

## Energy consumption per capita

Estimating the energy consumption per capita in space is not straightfoward as the astronauts onboard ISS do not live the same kind of life as we earthlings. However, we can imagine a future where space stations work very much like towns, with people living their everyday life like we do, in which case we could make an estimate for their consommation.

As a side note, in the US, the average energy consumption is 80,000 kWh per year per person. This value is higher than the world average.

To get the consumption in watts rather than kWh, we divide by the number of hours in a year (8760 hours) to get kW and multiply by 1000 to get W.

In [226]:
def kwh_to_watt(kwh: float) -> float:
    return kwh * 1000 / 8760

For the ISS, we will simply use an energy consumption per capita equal to the ratio of the total energy expanded by the number of crew. According to https://www.edn.com/international-space-station-iss-power-system/, the ISS uses 90kw of power to sustain its systems online, with a crew of 7 people in average.

In [227]:
energy_consumption_per_capita = 90_000 / 7

def get_total_population_consumption_watt(population: int) -> float:
    return float(population) * energy_consumption_per_capita

## Solar panel efficiency

In theory the ISS generates can generate up to 240kw of energy with its 2500m² of solar panels. However, given a solar energy flux of 1360 W/m², we could theoritically extract 3,400kw from 2500m² of solar panels.

This gives us an efficiency of 7% for the ISS solar panels. In theory, space based solar power can go up to about 40% efficiency, so this setting can be adjusted when taking into account future technologies.

In [228]:
solar_panel_efficiency = 0.07

Additionaly, the solar panels are not always lit by the sun. We can divide our efficiency by about 2.5 to account for the time the station is not facing the sun.

In [229]:
solar_panel_efficiency /= 2.5

## Energy recieved from Star

The energy flux emitted by the star can be obtained using Stefan-Boltzmann's law to compute the energy per unit of surface:

$$ \Phi_{star} = \sigma T^4 $$

Where $\sigma$ is the Stefan-Boltzmann constant, $5.67 \times 10^{-8} W m^{-2} K^{-4}$, and $T$ is the temperature of the star in Kelvin. For the Sun, $T = 5778 K$.

The result is expressed in $W m^{-2}$.

In order to get the total energy emitted by the star in Watts, we multiply the flux by the radius of the star.

In [230]:
import math

def get_total_star_energy_watt(star_temperature: float, star_radius: float) -> float:
    return 4 * math.pi * 5.67e-8 * star_radius ** 2 * star_temperature ** 4

The solar panel only recieves a fraction from this energy, we can compute the energy per unit of surface ($Wm^{-2}$) at a given distance of the star $D$ by dividing the total energy emitted by the surface of the sphere of radius $D$ (the energy is spread evenly on a larger sphere as you go further).

In [231]:
def get_energy_flux_received(distance: float, star_temperature: float, star_radius: float) -> float:
    return get_total_star_energy_watt(star_temperature, star_radius) / (4 * math.pi * distance ** 2)

## Putting it all together

Now that we have all of this, we can make an estimate of the solar panel surface area needed to power the ISS!

### Star settings

For this we will directly use the data from the sun

In [232]:
star_temperature = 5778
star_radius = 6.9634e8

### Station settings:

As ISS orbits around Earth, we can approximate its distance to the sun by using the distance between the Earth and the Sun. Additionaly the crew of the ISS is 7.

In [233]:
distance_to_star = 1.496e11
station_population = 7

Finally, we can compute the necessary solar panel area to power the station.

In [234]:
def compute_required_area(population: int, solar_panel_efficiency: float, distance_to_star: float, star_temperature: float, star_radius: float) -> float:
    energy_needed = get_total_population_consumption_watt(population)
    print(f"Energy needed: {energy_needed} W")

    energy_flux_received = get_energy_flux_received(distance_to_star, star_temperature, star_radius)
    print(f"Energy flux received from the star: {energy_flux_received} W/m^2")

    print(f"Solar panel efficiency: {solar_panel_efficiency}")

    area_needed = energy_needed / (energy_flux_received * solar_panel_efficiency)

    return area_needed

area_needed = compute_required_area(station_population, solar_panel_efficiency, distance_to_star, star_temperature, star_radius)

print(f"Area needed: {area_needed} m^2")

Energy needed: 90000.0 W
Energy flux received from the star: 1369.2183416438243 W/m^2
Solar panel efficiency: 0.028000000000000004
Area needed: 2347.5333455048385 m^2


According to https://www.nasa.gov/image-article/solar-arrays-international-space-station/, the ISS has 2500m² of solar panels. We are very close!

## Putting Nantes in space

Now we can make the calculation for a bigger station, the size of an entire city! I spent most of my childhood years in Nantes, a very nice city to the west of France. There is a huge debate to decide if Nantes is part of Bretagne or not, so I am going to settle the debate. Nantes is not in Bretagne, nor it is in Pays de la Loire. From now on Nantes will be in spaaaace!

For the sake of simplicity, we will assume a station in Earth orbit, so we will reuse the star parameters from earlier.

We only have to tweak the station population and the average consumption of its citizens. In France, the average energetic consumption is 40 000 kwh. As the station must power its life support, which is not necessary on Earth (yet!), we will increase than number to 50 000 kwh, which is a bit arbitrary but we have to start somewhere.

Additionnaly, we can have a higher efficiency with our solar panels (up to 40%) thanks to new advances in solar panel technology. We still need to account for time spend in the dark with the division by 2.5

In [235]:
station_population = 323_204
energy_consumption_per_capita = kwh_to_watt(50_000)
solar_panel_efficiency = 0.4 / 2.5


area_needed = compute_required_area(station_population, solar_panel_efficiency, distance_to_star, star_temperature, star_radius)

print(f"Area needed: {area_needed} m^2")

Energy needed: 1844771689.497717 W
Energy flux received from the star: 1369.2183416438243 W/m^2
Solar panel efficiency: 0.16
Area needed: 8420733.719881758 m^2


## Conclusion & Limitations

We now have a model to compute the solar panel area needed to sustain a given population in a space station. This model is not perfect because it does not account for energy efficiency inside the space station. On the ISS, this is not really an issue because the station is relatively small and its grid uses direct current.

On a larger station the size of Nantes, we should definitely take into account the energy efficiency of the grid: not 100% of the electricity produced will be available for the end users.

Overall this model is a good first step in order to create grounded procedural space stations in Cosmos Journeyer.